In [ ]:
%%capture systemMsg

!pip install suite2p
!pip install cellpose

import numpy as np
from skimage import io
import os, requests
from pathlib import Path
import suite2p
import matplotlib.pyplot as plt
from ScanImageTiffReader import ScanImageTiffReader
from PIL import Image


In [ ]:
#Concatenate scanimage raw files
filelist = []
for file in os.listdir():
    if file.endswith(".tif"):
        filelist.append(file)
filelist.sort(key=os.path.getmtime)
selection_tuple = tuple(filelist)


In [ ]:
#create `/registered` output folder
i = 0
while os.path.exists(os.getcwd() + '\\registered%s' % i): 
    i += 1
os.makedirs(os.getcwd() + '\\registered%s' % i)   
newdir = os.getcwd() + '\\registered%s' % i

In [ ]:
#for visual inspection of pixels in clipped image region used to measure background intensity
def drawTriangles(concatenated_input):

    plotTriangles = concatenated_input.copy()

    #upper left
    plotTriangles[:,:,
            np.triu_indices(25)[0] + 5,
            25 - np.triu_indices(25)[1] + 5
           ] = 1000
    #lower left
    plotTriangles[:,:,
            511 - np.triu_indices(16)[0] - 5,
            16 - np.triu_indices(16)[1] + 5
           ] = 1000
    #upper right
    plotTriangles[:,:,
            np.triu_indices(25)[0] + 5,
            511 - (25 - np.triu_indices(25)[1]) - 5 
           ] = 1000
    #lower right
    plotTriangles[:,:,
            511 - np.triu_indices(25)[0] - 5,
            511 - (25 - np.triu_indices(25)[1]) - 5
           ] = 1000
    
    plt.figure(figsize = (36,36))
    plt.subplot(2,2,1)
    plt.gca().set_title('ch0 image')
    plt.imshow(np.average(concatenated[:,0,:,:], axis = 0), vmin=-10, vmax=10)
    plt.subplot(2,2,2)
    plt.gca().set_title('ch0 triangles')
    plt.imshow(np.average(plotTriangles[:,0,:,:], axis = 0), vmin=-10, vmax=10)
    plt.subplot(2,2,3)
    plt.gca().set_title('ch1 image')
    plt.imshow(np.average(concatenated[:,1,:,:], axis = 0), vmin=-10, vmax=10)
    plt.subplot(2,2,4)
    plt.gca().set_title('ch1 triangles')
    plt.imshow(np.average(plotTriangles[:,1,:,:], axis = 0), vmin=-10, vmax=10)

In [ ]:
#measures background intensity in pixels at corner regions of image clipped by objective lens

def measureTriangles(concatenated_input): 
    
    #channel 0
    #upper left
    a1 = concatenated_input[:,0,
            np.triu_indices(25)[0] + 5,
            25 - np.triu_indices(25)[1] + 5
           ]
    #lower left
    a2 = concatenated_input[:,0,
            511 - np.triu_indices(16)[0] - 5,
            16 - np.triu_indices(16)[1] + 5
           ]
    #upper right
    a3 = concatenated_input[:,0,
            np.triu_indices(25)[0] + 5,
            511 - (25 - np.triu_indices(25)[1]) - 5 
           ]
    #lower right
    a4 = concatenated_input[:,0,
            511 - np.triu_indices(25)[0] - 5,
            511 - (25 - np.triu_indices(25)[1]) - 5
           ]
    global acat
    acat = np.concatenate((a1.flatten(),a2.flatten(),a3.flatten(),a4.flatten()))
   
    #channel 1
    #upper left
    b1 = concatenated_input[:,1,
            np.triu_indices(25)[0] + 5,
            25 - np.triu_indices(25)[1] + 5
           ]
    #lower left
    b2 = concatenated_input[:,1,
            511 - np.triu_indices(16)[0] - 5,
            16 - np.triu_indices(16)[1] + 5
           ]
    #upper right
    b3 = concatenated_input[:,1,
            np.triu_indices(25)[0] + 5,
            511 - (25 - np.triu_indices(25)[1]) - 5 
           ]
    #lower right
    b4 = concatenated_input[:,1,
            511 - np.triu_indices(25)[0] - 5,
            511 - (25 - np.triu_indices(25)[1]) - 5
           ]
    global bcat
    bcat = np.concatenate((b1.flatten(),b2.flatten(),b3.flatten(),b4.flatten()))
    
    print('SI_Ch2_Offset: ' + str(SI_Ch2_Offset) )
    print('SI_Ch3_Offset: ' + str(SI_Ch3_Offset) )
    
    print('channel 0 total pixel median: ' + str(np.median(acat)) )
    print('channel 0 upper left pixel median: ' + str(np.median( a1.flatten() )) )
    print('channel 0 lower left pixel median: ' + str(np.median( a2.flatten() )) )
    print('channel 0 upper right pixel median: ' + str(np.median( a3.flatten() )) )
    print('channel 0 lower right pixel median: ' + str(np.median( a4.flatten() )) )
    
    print('channel 1 total pixel median: ' + str(np.median(bcat)) )
    print('channel 1 upper left pixel median: ' + str(np.median( b1.flatten() )) )
    print('channel 1 lower left pixel median: ' + str(np.median( b2.flatten() )) )
    print('channel 1 upper right pixel median: ' + str(np.median( b3.flatten() )) )
    print('channel 1 lower right pixel median: ' + str(np.median( b4.flatten() )) )
    
    plt.figure(figsize=(16, 6)) 
    
    plt.subplot(2, 5, 1)
    plt.gca().set_title('ch0 total')
    plt.hist(acat, bins = 50, range = ( np.percentile(acat, 5)-10, np.percentile(acat, 95)+10 ))
    plt.subplot(2, 5, 2)
    plt.gca().set_title('ch0 upper left')
    plt.hist(a1.flatten(), bins = 50, range = ( np.percentile(a1.flatten(), 5)-10, np.percentile(a1.flatten(), 95)+10 ))
    plt.subplot(2, 5, 3)
    plt.gca().set_title('ch0 lower left')
    plt.hist(a2.flatten(), bins = 50, range = ( np.percentile(a2.flatten(), 5)-10, np.percentile(a2.flatten(), 95)+10 ))
    plt.subplot(2, 5, 4)
    plt.gca().set_title('ch0 upper right')
    plt.hist(a3.flatten(), bins = 50, range = ( np.percentile(a3.flatten(), 5)-10, np.percentile(a3.flatten(), 95)+10 ))
    plt.subplot(2, 5, 5)
    plt.gca().set_title('ch0 lower right')
    plt.hist(a4.flatten(), bins = 50, range = ( np.percentile(a4.flatten(), 5)-10, np.percentile(a4.flatten(), 95)+10 ))
    
    plt.subplot(2, 5, 6)
    plt.gca().set_title('ch1 total')
    plt.hist(bcat, bins = 50, range = ( np.percentile(bcat, 5)-10, np.percentile(bcat, 95)+10 ))
    plt.subplot(2, 5, 7)
    plt.gca().set_title('ch1 upper left')
    plt.hist(b1.flatten(), bins = 50, range = ( np.percentile(b1.flatten(), 5)-10, np.percentile(b1.flatten(), 95)+10 ))
    plt.subplot(2, 5, 8)
    plt.gca().set_title('ch1 lower left')
    plt.hist(b2.flatten(), bins = 50, range = ( np.percentile(b2.flatten(), 5)-10, np.percentile(b2.flatten(), 95)+10 ))
    plt.subplot(2, 5, 9)
    plt.gca().set_title('ch1 upper right')
    plt.hist(b3.flatten(), bins = 50, range = ( np.percentile(b3.flatten(), 5)-10, np.percentile(b3.flatten(), 95)+10 ))
    plt.subplot(2, 5, 10)
    plt.gca().set_title('ch1 lower right')
    plt.hist(b4.flatten(), bins = 50, range = ( np.percentile(b4.flatten(), 5)-10, np.percentile(b4.flatten(), 95)+10 ))
    
    plt.show()

In [ ]:
#read tifs, perform background subtraction, concatenate tifs

#first tif 
read_img = ScanImageTiffReader(selection_tuple[0])
concatenated =  np.reshape(read_img.data(), (int(read_img.data().shape[0]/2),2,512,512) ) #ScanImageTiffReader reads all images as (interleaved_frames, x, y). Reshape to (frames, ch, x, y)

#By default ScanImage performs Offset Correction using measured PMT voltages. See https://docs.scanimage.org/Configuration/PMT/index.html#pmt-calibration
#This correction is reversed by adding the ‘channelOffsets’ value from the .tif metadata to each image pixel
#Background subtraction is then accomplished using objective lens clipped pixels as described above 
SI_Ch2_Offset = int((read_img.metadata()).splitlines()[36][30:-1].split()[1]) #read ScanImage .tif metadata ‘channelOffsets’
SI_Ch3_Offset = int((read_img.metadata()).splitlines()[36][30:-1].split()[2])

concatenated[:,0,:,:] = concatenated[:,0,:,:] + SI_Ch2_Offset 
concatenated[:,1,:,:] = concatenated[:,1,:,:] + SI_Ch3_Offset
measureTriangles(concatenated)
concatenated[:,0,:,:] = concatenated[:,0,:,:] - int(np.median(acat)) 
concatenated[:,1,:,:] = concatenated[:,1,:,:] - int(np.median(bcat)) 
measureTriangles(concatenated)


#subsequent tifs
for i in range(1, len(selection_tuple)):
    read_img = ScanImageTiffReader(selection_tuple[i])
    reshaped = np.reshape(read_img.data(), (int(read_img.data().shape[0]/2),2,512,512))
    
    SI_Ch2_Offset = int((read_img.metadata()).splitlines()[36][30:-1].split()[1]) 
    SI_Ch3_Offset = int((read_img.metadata()).splitlines()[36][30:-1].split()[2])
    
    #subsequent tifs undo scanimage auto offset subtraction
    reshaped[:,0,:,:] = reshaped[:,0,:,:] + SI_Ch2_Offset 
    reshaped[:,1,:,:] = reshaped[:,1,:,:] + SI_Ch3_Offset
    measureTriangles(reshaped)
    #subsequent tifs subtract clipped pixels measured offsets
    reshaped[:,0,:,:] = reshaped[:,0,:,:] - int(np.median(acat)) 
    reshaped[:,1,:,:] = reshaped[:,1,:,:] - int(np.median(bcat)) 
    measureTriangles(reshaped)
    
    concatenated = np.concatenate((concatenated, reshaped))

drawTriangles(concatenated)

read_img.close() # Need to close the ScanImageTiffReader process


os.chdir(newdir)
text_file = open("raw_concat_inputs.txt", "w")
text_file.write('\n'.join(map(str, selection_tuple)))
text_file.close()
io.imsave('raw_concat.tif', concatenated)

In [ ]:
#set Suite2p parameters. See https://suite2p.readthedocs.io/en/latest/settings.html

ops = suite2p.default_ops()
ops['fs'] = 1.07 
ops['nchannels'] = 2 
ops['reg_tif']=1 #see https://github.com/MouseLand/suite2p/issues/89
ops['reg_tif_chan2'] = True
ops['two_step_registration'] = True
ops['keep_movie_raw'] = True
ops['max_overlap'] = 1
ops['roidetect'] = False
print(ops)

db = {
    'data_path': [os.getcwd()],
}
print(db)

output_ops = suite2p.run_s2p(ops=ops, db=db) #run
output_ops_file = np.load(Path(output_ops['save_path']).joinpath('ops.npy'), allow_pickle=True).item()
output_ops_file.keys() == output_ops.keys()

In [ ]:
#Concatenate outputs of Suite2p registration
def read_tiff(path):
    """
    path - Path to the multipage-tiff file
    """
    img = Image.open(path) #from PIL
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img, dtype=np.int16)) 
    return np.array(images)


os.chdir(newdir + '\\suite2p\\plane0\\reg_tif')
regtif = sorted(os.listdir(), key=os.path.getmtime)
regtif_concat = read_tiff(regtif[0])
for i in range(1, len(regtif)):
    regtif_concat = np.concatenate((regtif_concat, read_tiff(regtif[i])))  
os.chdir(newdir)
io.imsave('reg_concat_ch0.tif', regtif_concat)

os.chdir(newdir + '\\suite2p\\plane0\\reg_tif_chan2')
regtif_chan2 = sorted(os.listdir(), key=os.path.getmtime) #naming ch1 as `chan2` in code follwowing Suite2p convention. Output name as ch1 (ie FRET = ch1/ch0)
regtif_chan2_concat = read_tiff(regtif_chan2[0])
for i in range(1, len(regtif_chan2)):
    regtif_chan2_concat = np.concatenate((regtif_chan2_concat, read_tiff(regtif_chan2[i])))  
os.chdir(newdir)
io.imsave('reg_concat_ch1.tif', regtif_chan2_concat)

#save an avg project across slices and channels
fixed_sliceAvg_ch0 = np.average(regtif_concat, axis = 0).astype(np.float32)
fixed_sliceAvg_ch1 = np.average(regtif_chan2_concat, axis = 0).astype(np.float32)
fixed_sliceAvg_chAvg = ( (fixed_sliceAvg_ch0 + fixed_sliceAvg_ch1)/2 ) 
io.imsave('reg_concat_chAvg_sliceAvg.tif',fixed_sliceAvg_chAvg ) 

In [ ]:
#remove intermediate Suite2p files

os.remove('raw_concat.tif')
import shutil
shutil.rmtree('suite2p')

os.chdir("..") #move up one directory to folder containing input .tifs
#remove input files
for i in selection_tuple:
    if i in os.listdir(".."):
        os.remove(i)
